In [1]:
import pandas as pd
import numpy as np

In [16]:
num=1

In [2]:
def read_file(url):

    url = url + "?raw=true"
    df = pd.read_csv(url, encoding='cp1252', sep=',', low_memory=False)
    return df

In [3]:
def create_CI_table(iterations, queried_instances, queried_instances_uncertainty):
    pos_queried_d=[]
    pos_queried_u=[]
    pos_sample_d=[]
    pos_sample_u=[]

    for i in iterations:
        if i == 0:
            pos_queried_d.append(0)
            pos_queried_u.append(0)
            pos_sample_d.append(sample_init_pos.shape[0]/sample_init.shape[0])
            pos_sample_u.append(sample_init_pos.shape[0]/sample_init.shape[0])
        else:
            df_queried_d=df_train.iloc[list(queried_instances.iloc[:i,1])]
            df_queried_d_pos=df_queried_d.loc[df_queried_d['y_10']==1].shape[0]
            pos_queried_d.append(df_queried_d_pos/i)
            pos_sample_d.append((df_queried_d_pos+sample_init_pos.shape[0])/(i+sample_init.shape[0]))
            df_queried_u=df_train.iloc[list(queried_instances_uncertainty.iloc[:i,1])]
            df_queried_u_pos=df_queried_u.loc[df_queried_u['y_10']==1].shape[0]
            pos_queried_u.append(df_queried_u_pos/i)
            pos_sample_u.append((df_queried_u_pos+sample_init_pos.shape[0])/(i+sample_init.shape[0]))

    # assign data of lists.  
    data = {'i': iterations
            , '%_queried_d': pos_queried_d
            , '%_queried_u': pos_queried_u
            , '%_sample_d': pos_sample_d
            , '%_sample_u': pos_sample_u}  
    return pd.DataFrame(data) 

In [4]:
iterations=[0, 25, 50, 100, 250, 500, 750, 1000, 2500, 4000]
tables=[]

for num in range(10):
    df_train=read_file("https://github.com/hannahgathu/Thesis-Data-Visualisations/blob/main/Experiments/data/reduced_12000/CI10_bias1/train_CI10_bias1_{}.csv".format(num))
    df_test=read_file("https://github.com/hannahgathu/Thesis-Data-Visualisations/blob/main/Experiments/data/reduced_12000/test_{}.csv".format(num))
    
    sample_init=df_train.loc[df_train['in_sample']==1]
    sample_init_pos=sample_init.loc[sample_init['y_10']==1]

    queried_instances=pd.read_csv('./density/queried_instances_{}.csv'.format(num))
    queried_instances_uncertainty=pd.read_csv('./uncertainty/queried_instances_{}.csv'.format(num))
    
    CI_table=create_CI_table(iterations, queried_instances, queried_instances_uncertainty)  
    print(CI_table)
    tables.append(CI_table)

    
df_tables = pd.concat(tables)
mean_table = df_tables.groupby(level=0).mean()
print(mean_table)


      i  %_queried_d  %_queried_u  %_sample_d  %_sample_u
0     0      0.00000     0.000000    0.188742    0.188742
1    25      0.52000     0.320000    0.214067    0.198777
2    50      0.50000     0.400000    0.232955    0.218750
3   100      0.54000     0.540000    0.276119    0.276119
4   250      0.49200     0.520000    0.326087    0.338768
5   500      0.40200     0.538000    0.321696    0.406484
6   750      0.36000     0.517333    0.310837    0.423004
7  1000      0.33600     0.502000    0.301843    0.429339
8  2500      0.24200     0.318400    0.236260    0.304425
9  4000      0.19525     0.233000    0.194793    0.229893
      i  %_queried_d  %_queried_u  %_sample_d  %_sample_u
0     0      0.00000     0.000000    0.182119    0.182119
1    25      0.48000     0.480000    0.204893    0.204893
2    50      0.60000     0.500000    0.241477    0.227273
3   100      0.52000     0.520000    0.266169    0.266169
4   250      0.50000     0.568000    0.326087    0.356884
5   500      0

In [5]:
mean_table.to_csv('b1_CI_table.csv')